In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [22]:
%pwd

'/Users/harshbhatt/Desktop/Projects/walmart-sales-forecast-mlops'

In [4]:
os.chdir("../")

In [30]:
from projectFiles.constants import *
from projectFiles.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
import zipfile
from projectFiles import logger
from projectFiles.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_URL, filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded with the following info: {headers}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip(self):
        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()
except Exception as e:
    raise e

[2025-03-01 14:57:42,062: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-01 14:57:42,063: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-01 14:57:42,064: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-01 14:57:42,065: INFO: common: created directory at: artifacts]
[2025-03-01 14:57:42,065: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-01 14:57:44,148: INFO: 1943541985: artifacts/data_ingestion/data.zip downloaded with the following info: Connection: close
Content-Length: 3568662
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "05d8a13cbf046f4589e7006c96086f75acd109c3a706c5e56299b353b7d38c0d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1D3C:2147D:1663A0:1C43F1:67C366B4
Accept-Ranges: bytes
Date: Sa